In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- Configuration ---
DATASET_DIR = 'dataset'
IMG_SIZE = (100, 100)
# --- End Configuration ---

def load_data():
    """
    Loads all pre-cropped images from the 'dataset' folder.
    """
    print("Loading image data from dataset...")
    data = []
    labels = []
   
    for person_name in os.listdir(DATASET_DIR):
        person_dir = os.path.join(DATASET_DIR, person_name)
        if not os.path.isdir(person_dir):
            continue
           
        for img_file in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_file)
           
            try:
                # --- NEW: Load image in COLOR ---
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
               
                if img is None:
                    print(f"Warning: Could not read image {img_path}. Skipping.")
                    continue
               
                if img.shape[0:2] != IMG_SIZE:
                    img = cv2.resize(img, IMG_SIZE)
                   
                data.append(img)
                labels.append(person_name)
               
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")

    print(f"Loaded {len(data)} images.")
    if len(data) == 0:
        print("Error: No data loaded. Is the 'dataset' folder empty?")
        exit()

    # --- Preprocessing ---
   
    # 1. Convert lists to NumPy arrays
    data = np.array(data)
    labels = np.array(labels)

    # 2. --- NEW: Normalize pixel values (from 0-255 to 0.0-1.0) ---
    # This is the standard normalization for custom CNNs
    data = data.astype('float32') / 255.0
   
    # 3. Encode text labels (e.g., "Priya") into numbers (e.g., 0)
    le = LabelEncoder()
    labels_encoded = le.fit_transform(labels)
   
    # 4. One-Hot Encode the number labels
    num_classes = len(np.unique(labels_encoded))
    labels_one_hot = to_categorical(labels_encoded, num_classes=num_classes)
   
    # 5. Save the label encoder classes (the names)
    np.save('labels.npy', le.classes_)
    print(f"Saved label mapping (e.g., 0={le.classes_[0]}): labels.npy")
   
    # 6. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        data, labels_one_hot, test_size=0.2, random_state=42, stratify=labels_one_hot
    )
   
    print(f"Training data shape: {X_train.shape}")
    print(f"Testing data shape: {X_test.shape}")
   
    return X_train, X_test, y_train, y_test, num_classes

def build_custom_cnn(num_classes):
    """
    Builds a more powerful custom CNN model.
    """
    model = Sequential()
   
    # Input layer - now expects (100, 100, 3) for color images
    model.add(Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))

    # --- Block 1 ---
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # --- Block 2 ---
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # --- Block 3 ---
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # --- Classifier Head ---
    model.add(Flatten())
   
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
   
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
   
    # --- Output Layer ---
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    return model

# --- Main Training Script ---

# 1. Load and preprocess the data
X_train, X_test, y_train, y_test, num_classes = load_data()

if num_classes < 2:
    print("Error: Need at least 2 different people in the 'dataset' folder to train.")
    exit()

# 2. Build the new custom model
model = build_custom_cnn(num_classes)

# 3. Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 4. --- NEW: Set up Data Augmentation ---
datagen = ImageDataGenerator(
    rotation_range=20,     # Randomly rotate images
    zoom_range=0.15,       # Randomly zoom in
    width_shift_range=0.2, # Randomly shift horizontally
    height_shift_range=0.2,# Randomly shift vertically
    shear_range=0.15,      # Shear transformations
    horizontal_flip=True,  # Randomly flip horizontally
    fill_mode="nearest"
)

# 5. Train the model
print("\n--- Starting Model Training ---")
# Increase epochs because augmentation makes the task harder
epochs = 75
batch_size = 32 # We have very few base images, so a small batch size is good

# Use datagen.flow() to feed augmented images
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size if len(X_train) > batch_size else 1,
    epochs=epochs,
    validation_data=(X_test, y_test)
)
print("--- Training Finished ---")

# 6. Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

# 7. Save the trained model
model.save('face_model_cnn.keras')
print("Trained model saved as: face_model_cnn.keras")

Loading image data from dataset...
Loaded 264 images.
Saved label mapping (e.g., 0=Navraj): labels.npy
Training data shape: (211, 100, 100, 3)
Testing data shape: (53, 100, 100, 3)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_57 (Conv2D)              │ (None, 100, 100, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 100, 100, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_52          │ (None, 100, 100, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_43 (Dropout)            │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 50, 50, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_60 (Conv2D)              │ (None, 50, 50, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_53          │ (None, 50, 50, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_34 (MaxPooling2D) │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_61 (Conv2D)              │ (None, 25, 25, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_62 (Conv2D)              │ (None, 25, 25, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_54          │ (None, 25, 25, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_35 (MaxPooling2D) │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 256)            │     4,718,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_55          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_46 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_56          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_47 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,046,860 (19.25 MB)

 Trainable params: 5,045,644 (19.25 MB)

...


--- Starting Model Training ---


c:\Users\admin\anaconda3\envs\prlenv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/75
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 795ms/step - accuracy: 0.0056 - loss: 5.0481 - val_accuracy: 0.0377 - val_loss: 3.7865
Epoch 2/75
1/6 ━━━━━━━━━━━━━━━━━━━━ 3s 609ms/step - accuracy: 0.0000e+00 - loss: 5.1723

c:\Users\admin\anaconda3\envs\prlenv\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.0000e+00 - loss: 5.1723 - val_accuracy: 0.0377 - val_loss: 3.7915
Epoch 3/75
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 693ms/step - accuracy: 0.0447 - loss: 4.4379 - val_accuracy: 0.0189 - val_loss: 3.8379
Epoch 4/75
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.0312 - loss: 4.5304 - val_accuracy: 0.0189 - val_loss: 3.8713
Epoch 5/75
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 659ms/step - accuracy: 0.0726 - loss: 4.1404 - val_accuracy: 0.0189 - val_loss: 4.1581
Epoch 6/75
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.0312 - loss: 4.1846 - val_accuracy: 0.0000e+00 - val_loss: 4.1948
Epoch 7/75
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 649ms/step - accuracy: 0.0838 - loss: 3.8918 - val_accuracy: 0.0000e+00 - val_loss: 4.3478
Epoch 8/75
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.0625 - loss: 4.0226 - val_accuracy: 0.0000e+00 - val_loss: 4.3393
Epoch 9/75
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 673ms/step - accuracy: 0.0447 - loss: 3.9858 - val_accuracy: 0.0000e+00 - val_